# Memory based algorithm for recommendation

In this notebook we will write a collaborative filtering algorithm for a recommendation problem. 

The MovieLens dataset (ml-latest-small) describes 5-star rating and free-text tagging activity from MovieLens, a movie recommendation service. It contains 100004 ratings and 1296 tag applications across 9125 movies. https://grouplens.org/datasets/movielens/. To get the a subset of the data:

`wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip`

Here is a simple recommendation algorithm.
1. Compute similarity between movies (using say Jaccard similarity).
2. Given a movie j, return the top K similar movies.

## MovieLens dataset

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

In [2]:
PATH = Path("ml-latest-small")
list(PATH.iterdir())

[PosixPath('ml-latest-small/links.csv'),
 PosixPath('ml-latest-small/tags.csv'),
 PosixPath('ml-latest-small/ratings.csv'),
 PosixPath('ml-latest-small/README.txt'),
 PosixPath('ml-latest-small/movies.csv')]

In [3]:
! head ml-latest-small/ratings.csv

In [4]:
# reading a csv into pandas
data = pd.read_csv(PATH/"ratings.csv")

In [5]:
data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


## Splitting training and validation

In [6]:
time_80 = np.quantile(data.timestamp.values, 0.8)
time_80

1458635171.0

In [7]:
train = data[data["timestamp"] < time_80].copy()
val = data[data["timestamp"] >= time_80].copy()

In [8]:
val.head()

,userId,movieId,rating,timestamp
1434,15,1,2.5,1510577970
1436,15,47,3.5,1510571970
1440,15,260,5.0,1510571946
1441,15,293,3.0,1510571962
1442,15,296,4.0,1510571877


In [9]:
movies = train.movieId.unique()
len(movies)

7867

## Jaccard similarity
Write a function that computes the jacard similarity between two movies.

In [10]:
def Jaccard_similarity(train, j1, j2):
    """ compute Jaccard similarity between two movies
    
    1. comptute the set of users that have rated j1 and j2 respectively 
        (call it m1 and m2)
    2. return |m1 intercept m2|/ |m1 union m2|
    
    hints: use functions np.union1d and np.intersect1d
    """
    # YOUR CODE HERE

In [29]:
j1 = 1
j2 = 47 
Jaccard_similarity(train, j1, j2)

0.3007246376811594

In [34]:
len(movies)

7867

In [38]:
movies = train.movieId.unique()
movie2id = {v:k for k,v in enumerate(movies)}
#movie2id

In [11]:
def K_nearest_neighbor(train, movies, j1, K):
    """ Returns K top similar movies to j1.
    
    Compute similarities between j1 and all other movies.
    Returns top K movies.
    (Hint use np.argsort)
    """
    # YOUR CODE HERE

## Making predictions

Let's look for example at user with id 15. Let's find the last movie that this user had in the training data and make predictions.

In [12]:
train_15 = train[train["userId"] == 15]
train_15.sort_values(by=['timestamp']).iloc[-10:]

,userId,movieId,rating,timestamp
1444,15,355,1.0,1299425002
1439,15,256,3.0,1299425021
1472,15,2150,5.0,1299425040
1452,15,849,2.0,1299425064
1483,15,3510,5.0,1299425097
1491,15,4018,4.0,1299425113
1462,15,1347,3.0,1299425144
1487,15,3617,1.5,1299425200
1485,15,3555,4.0,1299425208
1526,15,69757,4.0,1299425345


In [81]:
K = 40
j1 = 69757
top = K_nearest_neighbor(train, movies, j1, K)

In [82]:
top

array([69757, 72011, 74789, 76251, 51540, 88163, 69122, 79702, 64839,
       80549, 89492, 76077, 78499, 87232,  7454, 46972, 69844, 34150,
       91077,  4018, 62434, 31685, 57669, 77561, 88129, 67734, 59784,
       61323, 70286, 98809, 91542, 73017, 53993, 86882, 64969, 60074,
       81591, 63131, 86833, 60756])

Let's compare that list with the list of movies user 15 watched in the validation set.

In [83]:
val_15 = val[val["userId"] == 15]
val_15.movieId.values

array([     1,     47,    260,    293,    296,    318,    356,    364,
          527,    588,    589,    596,    780,    858,   1196,   1198,
         1200,   1210,   1214,   1240,   1265,   1270,   1527,   1653,
         2011,   2012,   2028,   2081,   2085,   2329,   2571,   2762,
         2858,   2916,   2959,   3147,   3156,   3499,   3535,   3578,
         3753,   3949,   3994,   4022,   4226,   4306,   4370,   4720,
         4886,   4993,   4995,   5445,   5618,   5952,   5971,   5989,
         6377,   6502,   6874,   7254,   7438,   8360,   8644,   8961,
        33493,  48304,  48774,  48780,  50872,  56174,  58559,  59315,
        60069,  63859,  64614,  68237,  68954,  70286,  71057,  71264,
        72998,  79132,  84152,  84954,  85414,  89745,  91500,  91529,
        94864,  96610,  97938,  99114, 101864, 103249, 104841, 105504,
       109374, 109487, 110102, 111759, 112556, 112852, 115149, 115713,
       120466, 122882, 122886, 122904, 122922, 122924, 134130, 134853,
      

In [14]:
# here is one way to evalute the algorithm
#np.intersect1d(val_15.movieId.values, top)

In [6]:
# try also j1 = 3510

## Approximate nearest neighbour (bonus)

Use the nmslib library to find approximate nearest neighbours

here is an example on how to use the nmslib library: <br>
https://github.com/benfred/bens-blog-code/blob/master/approximate_als/test_batch_queries.py

In [ ]:
#import nmslib